# Building the UI
## Before we start (OPTIONAL)
Before we start, it might be a good idea to fork this repository. This is to ensure that we can easily get a link to your schema.

![fork](./img/fork.png)

Once you forked the repo, you can then clone your own repository.

## Introduction
In addition to the Neo4j console and the notebook earlier, we can provide develop a public facing website that we can share with the public without providing them write access to the neo4j database.

The UI provides single and two-term search where the latter uses allShortestPath to query all the shortest paths between two terms. To build the UI, we need to create a UI schema that is built using the data that was ingested in the graph db. This notebook automates the creation of this schema file.

In [1]:
import re
import json
from glob import glob
import pandas as pd

# Protocol 4 - 2

In [2]:
# Make sure to include the final / otherwise this will fail.
path_to_network = "../construction/Network/"

## Change the path to your network.

# Protocol 4 - 3

## Node Schema

```

{
    "node": "Publications",
    "example": [|
        38648692,
        39089348
    ],
    "display": [
        {
            "label": "label",
            "text": "${label}",
            "type": "text"
        },
        {
            "label": "Title",
            "text": "${Title}",
            "type": "text"
        },
        {
            "label": "Authors",
            "text": "${Authors}",
            "type": "text"
        },
        {
            "label": "Journal",
            "text": "${Journal}",
            "type": "text"
        },
        {
            "label": "Publications ID",
            "text": "${id}",
            "type": "text"
        }
    ],
    "relation": [],
    "search": [
        "id",
        "label"
    ],
    "color": "#ADB2D4"
}


```
This is a node schema for the node "Publications". The example field will be displayed display as an example on the UI while the display field tells the UI what to display as a tooltip. Meanwhile, search tells the UI that we can search both id and label fields of the nodes. Node colors are automatically generated but can otherwise be set using the field color for more customization.

In [3]:
node_schema = []
node_pattern = "(?P<directory>.+)/(?P<label>.+)\.(?P<entity>.+)\.csv"
color_palette = ["#ADB2D4", "#C7D9DD", "#D5E5D5", "#EEF1DA"]
counter = 0
for filename in glob(path_to_network + "*.nodes.csv"):
    match = re.match(node_pattern, filename).groupdict()
    label = match["label"].replace("_", " ")
    df = pd.read_csv(filename)
    display = []
    for i in df.columns:
        if i == "id":
            display.append({
                'label': '%s ID'%label,
                'text': "${%s}"%i,
                'type': 'text'
            })
        else:
            val = str(df.at[0, i])
            if val.startswith("http"):
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'href': "${%s}"%i,
                    'type': 'link'
                })
            else:
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'type': 'text'
                })
    examples = [str(x) for x in list(df.head(2).label)]
    color = color_palette[counter % len(color_palette)]
    node = {
        "node": label,
        "example": examples,
        "display": display,
        "relation": [],
        "search": [
            "label"
        ],
        "color": color
    }
    node_schema.append(node)
    counter += 1
        

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/4l/73ssnfdj1mg_zmt5ghdmqjjh0000gp/T/ipykernel_64273/4013982737.py:2: SyntaxWarning: invalid escape sequence '\.'
  node_pattern = "(?P<directory>.+)/(?P<label>.+)\.(?P<entity>.+)\.csv"


# Protocol 4 - 4

## Edge Schema
```
{
    "match": [
        "Publications"
    ],
    "selected": true,
    "color":"#C7D9DD",
    "directed": "triangle",
    "display": [
        {
            "label": "target label",
            "text": "${target_label}",
            "type": "text"
        },
        {
            "label": "source label",
            "text": "${source_label}",
            "type": "text"
        },
        {
            "label": "relation",
            "text": "${relation}",
            "type": "text"
        },
        {
            "label": "source",
            "text": "${source}",
            "type": "text"
        },
        {
            "label": "target",
            "text": "${target}",
            "type": "text"
        }
    ]
}
```

This is the same with node schema with a few differences, first match allows you to match multiple edge types and selected is a field that allows us to select what edges are queried by default when doing two term searches.

In [4]:
edge_schema = []
edge_pattern = "(?P<directory>.+)/(?P<source_type>.+)\.(?P<relation>.+)\.(?P<target_type>.+)\.(?P<entity>.+)\.csv"

for filename in glob(path_to_network + "*.edges.csv"):
   print(filename)
   match = re.match(edge_pattern, filename).groupdict()
   entity = match["entity"]
   source_type = match["source_type"]
   relation = match["relation"].replace("_", " ")
   edge = {
      "match": [relation],
      "selected": True,
   }
   df = pd.read_csv(filename)
   display = []
   for i in df.columns:
      if i == "id":
         display.append({
               'label': '%s ID'%label,
               'text': "${%s}"%i,
               'type': 'text'
         })
      else:
         val = str(df.at[0, i])
         if val.startswith("http"):
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'href': "${%s}"%i,
                  'type': 'link'
               })
         else:
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'type': 'text'
               })
         edge["display"] = display
   edge_schema.append(edge)
         

../construction/Network/Authors.Publications.Publications.edges.csv
../construction/Network/Publications.Awards.Awards.edges.csv
../construction/Network/MeSH.MeSH.Publications.edges.csv
../construction/Network/Awards.Awards.Publications.edges.csv
../construction/Network/Awards.Awards.Authors.edges.csv
../construction/Network/Authors.Awards.Awards.edges.csv
../construction/Network/Publications.Publications.Authors.edges.csv
../construction/Network/Publications.MeSH.MeSH.edges.csv
../construction/Network/Authors.Coauthors.Authors.edges.csv


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/4l/73ssnfdj1mg_zmt5ghdmqjjh0000gp/T/ipykernel_64273/3187522818.py:2: SyntaxWarning: invalid escape sequence '\.'
  edge_pattern = "(?P<directory>.+)/(?P<source_type>.+)\.(?P<relation>.+)\.(?P<target_type>.+)\.(?P<entity>.+)\.csv"


# Protocol 4 - 5

## Header Schema


The next part is defining the header portion of the schema that you can use to modify our UI. Notice the tabs field, this is where we define the pages in our knowledge graph as well as the components that you want to render.

Currently we have the following components available:
1. KnowledgeGraph: this renders the single and two term search components
2. Markdown: this is for rendering static markdown files, useful for tutorials
3. Enrichment: For an enrichment page.
   
Each component takes specific props like init_function that initializes/prefetches the static props for the page. For this example, we only look at the KnowledgeGraph component. For more information on how to use the other components you can check: https://maayanlab.cloud/enrichr-kg/api/knowledge_graph/schema.

In [5]:
# First you need to modify this based on your dataset. 
# This will be the initial query that the users will see when visiting your site
initial_query = {
	"start": "Authors",
	"start_term": "Robert O Wright",
	"start_field": "label"
}

title = "Demo-KG"
icon = "https://minio.dev.maayanlab.cloud/enrichr-kg/enrichr-kg.png"
header = {
		"header": title,
		"title": title,
		"background": {
			"backgroundColor": "#000",
			"contrastText": "#FFF"
		},
		"icon": {
			"src": icon,
			"favicon": icon,
			"faviconTitle": title,
			"alt": title,
			"key": "%s_logo"%title,
			"width": 100,
			"height": 60
		},
		"tabs": [
			{
				"endpoint": "/",
				"label": "Term & Gene Search",
				"type": "page",
				"component": "KnowledgeGraph",
				"props": {
					"init_function": "initialize_kg",
					"initial_query": initial_query,
					"subheader": {
						"url_field": "filter",
						"query_field": "relation"
					}
				}
				}
		]
}

# Protocol 4-7


## Footer Schema

The next part is the footer portion of the schema that you can use to modify our UI. The footer contains three different fields "style","layout", and "footer_text". The style field defines the background and font color, whereas layout is an array of arrays where the outer array indicates the column of the footer, and the inner array(s) indicates the rows of the footer. Within the layout you can easily declare different components like logos, links, etc. The "footer_text" field allows for creation of text in the footer.

In [6]:
footer = {
		"style": {
            "background": "#000",
            "color": "#FFF"
        },
		"layout": [
			[
				{
                    "component": "logo",
                    "props": {
                        "src": "https://minio.dev.maayanlab.cloud/enrichr-kg/enrichr-kg.png",
                        "alt": "enrichr-kg",
                        "title": "Demo-KG",
                        "color": "inherit",
                        "size": "small"
                    }
                },
				{
                    "component": "github",
                    "props": {
                        "code": "https://github.com/MaayanLab/Knowledge-Graph-UI",
                        "issues": "https://github.com/MaayanLab/Knowledge-Graph-UI/issues/new"
                    }
                }
			],
			[
				{
					"component": "icon",
					"props": {
						"type": "icon",
						"key": "lab_icon",
						"src": "https://minio.dev.maayanlab.cloud/enrichr-kg/maayanlab.png",
						"alt": "MaayanLab",
						"href": "https://labs.icahn.mssm.edu/maayanlab/",
						"height": 60,
						"width": 150
					}
				  }
			],
			[
				{
					"component": "icon",
					"props": {
						"type": "icon",
						"key": "center_icon",
						"src": "https://minio.dev.maayanlab.cloud/enrichr-kg/bioinformatics.png",
						"alt": "COB",
						"href": "https://icahn.mssm.edu/research/bioinformatics",
						"height": 60,
						"width": 150
					}
				  }
			]
		],
		"footer_text": "<span style='color:white'>Please acknowledge Enrichr-KG in your publications by citing the following reference:<br><a style='color: cyan' href='https://doi.org/10.1093/nar/gkad393'>Evangelista JE, Xie Z, Marino GB, Nguyen N, Clarke DJB, Ma'ayan A. Enrichr-KG: bridging enrichment analysis across multiple libraries. Nucleic Acids Res. 2023 May 11:gkad393. doi: 10.1093/nar/gkad393. PMID: 37166973.</a></span>"
}

# Protocol 4 - 8

## Define the theme and put together the whole schema.

Here we declare the UI theme for the schema and concatenate and join the header and footer to write out the whole schema.

In [7]:

rest = {
	"header": header,
	"footer": footer,
	"ui_theme": "enrichr_kg_theme"
}

## Putting it all together

In [8]:
schema = {
    "nodes": node_schema,
    "edges": edge_schema,
    **rest
}

with open("../schema_sample.json", 'w') as o:
    o.write(json.dumps(schema, indent=4))

Once you have the schema you can upload it somewhere to obtain a link of the file. If you forked the repo you can commit it and push to your forked repository.

```
git add schema.json
git commit -m updated schema
git push
```

To get the link, go to schema.json on github and click raw:

![raw](./img/raw.png)
![raw_link](./img/raw_link.png)